In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from torch.utils.data import Dataset
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch

train_dataset_name = 'ArithOpsTrain.xlsx'
df = pd.read_excel(train_dataset_name)
df = df.drop('Table 1',axis=1)
df = df.rename(columns=df.iloc[0]).loc[1:]

train_df , valid_df = train_test_split(df,test_size=0.1,random_state=0)

/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
train_df.head()

,Description,Question,Equation,Input Numbers,Output
373,mrs. hilt is baking bread . she needs number0 ...,how much flour will she need to make number2 l...,/ number0 number1,5 2 1,2.5
901,robin had number0 songs on her number1 player ...,how many songs does she have on her number4 pl...,+ - number0 number2 number3,30 3 8 10 3,32
254,there are number0 more sections that are undev...,what is the total area of the undeveloped land ?,* number0 number1,3 2435,7305
468,mom made number0 chocolate chip cookies . it t...,how many cookies were left ?,- number0 number3,32 24 16 9,23
333,number0 children are taking a bus to the zoo ....,how many seats will the children need in all ?,/ number0 number1,58 2,29


In [4]:
class T5Dataset(Dataset):
    def __init__(
        self,
        data  : pd.DataFrame,
        tokenizer : T5Tokenizer,
        text_max_token_length = 512 ,
        output_max_token_length = 128
    ):
        
        super().__init__()
        self.tokenizer = tokenizer
        self.data = data 
        self.text_max_token_length = text_max_token_length
        self.output_max_token_length = output_max_token_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        data_row = self.data.iloc[index]

        input_text = data_row["Description"]
        input_question = data_row["Question"]

        in_text = input_text + " [SEP] " + input_question
        
        input_text_encoding = self.tokenizer(
            in_text,
            max_length=self.text_max_token_length,
            padding = "max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        

        output_text = data_row["Equation"]        
        output_text_encoding = self.tokenizer(
            output_text,
            max_length=self.output_max_token_length,
            padding = "max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        input_numbers_text = data_row["Input Numbers"]
        in_numbers = [float(x) for x in input_numbers_text.split(' ')]


        final_result  = float(data_row["Output"])

        return dict(
            input_text = input_text,
            output_text = output_text,
            input_text_ids = input_text_encoding['input_ids'].flatten(),
            input_attention_mask = input_text_encoding['attention_mask'].flatten(),
            output_text_ids = output_text_encoding['input_ids'].flatten(),
            output_attention_mask = output_text_encoding['attention_mask'].flatten(),
                
            input_numbers  = in_numbers,
            final_result = final_result

        )   

In [5]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
special_tokens_dict = {'additional_special_tokens' : ['[SEP]']}
num_added_tokens = t5_tokenizer.add_special_tokens(special_tokens_dict)

dataset = T5Dataset(train_df,t5_tokenizer)

/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
from torch.utils.data import DataLoader

class T5DataSetModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df : pd.DataFrame,
        valid_df : pd.DataFrame,
        tokenizer : T5Tokenizer,
        batch_size = 32,
        text_max_token_length = 512,
        output_max_token_length = 128
    ):
        super().__init__()
        self.train_df = train_df
        self.valid_df = valid_df
        self.tokenizer = tokenizer,
        self.batch_size = batch_size,
        self.text_max_token_length = text_max_token_length
        self.output_max_token_length = output_max_token_length

    
    def setup(self, stage = None):
        
        self.train_dataset = T5Dataset(self.train_df,self.tokenizer,self.text_max_token_length,self.output_max_token_length)
        self.valid_dataset = T5Dataset(self.valid_df,self.tokenizer,self.text_max_token_length,self.output_max_token_length)
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            self.batch_size,
            shuffle=True,
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.valid_dataset,
            self.batch_size,
            shuffle=False
        )
#data_module = T5DataSetModule(train_df,valid_df,t5_tokenizer,BATCH_SIZE)       

In [7]:
def postfix_evaluation(batch_data,input_values):

    arith_symbols = set(['+','-','*','/','%'])
    output_values = []
    
    for i in range(len(batch_data)):
        flag = True
        current_input = batch_data[i].split(' ')
        current_input.reverse()
        input_value = input_values[i]

        stack = []
        for symbol in current_input:
            if symbol in arith_symbols:
                if len(stack)<2:
                    flag = False
                    break
                in1 = stack.pop(-1)
                in2 = stack.pop(-1)

                res = 0
                if symbol=='+':
                    res = in1+in2
                elif symbol=='-':
                    res = in1 - in2 
                elif symbol == '*':
                    res = in1 * in2
                elif symbol=='/':
                    res = in1/in2
                else:
                    res = in1 % in2
                stack.append(res)


            else:
                if "number" in symbol:
                    index = int(symbol[6])
                    stack.append(input_value[index])

        if flag==False or len(stack)!=1:
            output_values.append(0)
        else:
            output_values.append(stack.pop(-1))
    

    return torch.tensor(output_values)

ans = postfix_evaluation(["+ - number0 number1 number2","+ / - number0 number2 number1 number3"],[[1.0,4.0,6.0],[5.0,6.0,7.0,8.0]])

In [9]:
ans

tensor([3.0000, 7.6667])

In [ ]:
import torch.optim as optim
class T5ArithTranslator(pl.LightningModule):

    def __init__(self,t5_tokenizer,weighing_factor=0.001):
        super().__init__()
        self.t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
        self.weighing_factor = weighing_factor
        self.tokenizer = t5_tokenizer

    def forward(self, input_ids, input_attention_mask, decoder_attention_mask, labels):

        outs = self.t5_model(input_ids=input_ids,attention_mask = input_attention_mask,labels = labels)        
        return outs.loss ,  outs.logits

        
    def training_step(self, batch, batch_idx) :
        
        input_text_ids = batch["input_text_ids"]
        input_attention_mask = batch["input_attention_mask"]
        output_text_ids = batch["output_text_ids"]
        output_attention_mask = batch["output_attention_mask"]

        loss, outs = self(
            input_text_ids,
            input_attention_mask,
            output_attention_mask,
            output_text_ids
        )

        preds = F.softmax(outs, dim=-1).argmax(dim=-1)
        y = self.tokenizer.batch_decode(sequences=preds, skip_special_tokens=True)


        print(len(y))
        self.log("train_loss" , loss, prog_bar=True,logger=True)
        return loss 
    
    def validation_step(self, batch, batch_idx):
        
        input_text_ids = batch["input_text_ids"]
        input_attention_mask = batch["input_attention_mask"]
        output_text_ids = batch["output_text_ids"]
        output_attention_mask = batch["output_attention_mask"]

        loss, outs = self(
            input_text_ids,
            input_attention_mask,
            output_attention_mask,
            output_text_ids
        )

        self.log("valid_loss" , loss, prog_bar=True,logger=True)
        return loss 

    def configure_optimizers(self):
        return optim.Adam(self.parameters(),lr = 0.0001)


In [ ]:
N_EPOCHS = 300
BATCH_SIZE = 32

model = T5ArithTranslator(t5_tokenizer)

checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename="best-checkpoint",
    save_top_k = 1,
    verbose = True,
    monitor="valid_loss",
    mode = "min"
)

logger = TensorBoardLogger("lightning_logs",name="translator")

trainer = pl.Trainer(
    logger = logger,
    callbacks =  checkpoint_callback,
    max_epochs=N_EPOCHS,
    log_every_n_steps=5,
    gpus=1,
    accelerator='cpu'
)

In [ ]:

def collate_function(batch_data):

    input_text_ids_list = [  batch["input_text_ids"] for batch in batch_data ] 
    input_attention_mask_list = [ batch["input_attention_mask"] for batch in batch_data ] 
    output_text_ids_list = [ batch["output_text_ids"] for batch in batch_data ] 
    output_attention_mask_list = [batch["output_attention_mask"] for batch in batch_data]

    input_numbers = [ batch["input_numbers"] for batch in batch_data ]
    result_list = [batch["final_result"] for batch in batch_data]

    
    input_text_ids = torch.stack(input_text_ids_list)
    input_attention_mask = torch.stack(input_attention_mask_list)
    output_text_ids = torch.stack(output_text_ids_list)
    output_attention_mask = torch.stack(output_attention_mask_list)

    return {
        "input_text_ids" : input_text_ids,
        "input_attention_mask" : input_attention_mask,
        "output_text_ids" : output_text_ids,
        "output_attention_mask" : output_attention_mask,
        "input_numbers" : input_numbers,
        "final_result" : result_list
    }


    


train_dataset = T5Dataset(train_df,t5_tokenizer)
valid_dataset = T5Dataset(valid_df,t5_tokenizer)

train_dataloader = DataLoader(train_dataset,32,True,collate_fn= collate_function)
valid_dataloader = DataLoader(valid_dataset,32,shuffle=False,collate_fn=collate_function)

In [ ]:
trainer.fit(model,train_dataloader,valid_dataloader)

### Inference Model

In [ ]:
test_model = T5ArithTranslator.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
test_model.freeze()

t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
special_tokens_dict = {'additional_special_tokens' : ['[SEP]']}
num_added_tokens = t5_tokenizer.add_special_tokens(special_tokens_dict)


In [ ]:
test_input_ids = t5_tokenizer("last stop in their field trip was the aquarium . penny identified number0 species of sharks number1 species of eels and number2 different species of whales . [SEP] how many species was penny able to identify ?",return_tensors='pt').input_ids

In [ ]:
outputs = test_model.t5_model.generate(test_input_ids)
print(t5_tokenizer.decode(outputs[0], skip_special_tokens=True))